In [ ]:
import numpy as np
from matplotlib import pyplot as plt

In [ ]:
RANDOM_SEED = 0x0

# TASK 1 (4 Points): K-Means using scikit-learn

First we will generate some data. The variable `X` will contain the data used in this section.

In [ ]:
from sklearn.datasets import make_blobs

In [ ]:
X, _ = make_blobs(
    n_samples=300,
    centers=4,
    cluster_std=.6,
    random_state=RANDOM_SEED,
)

plt.scatter(X[:, 0], X[:, 1], s=50)

### Task 1a

Cluster the above data using the K-Means implementation provided by scikit-learn.
Refer to the official documentation: <https://scikit-learn.org/stable/modules/generated/sklearn.cluster.KMeans.html>

* create an instance of the class `sklearn.cluster.KMeans`
* choose a suitable value for the `n_clusters` parameter
* call the `.fit` method to compute the clustering
* call the `.predict` method to get cluster labels for each data point

In [ ]:
from sklearn.cluster import KMeans

In [ ]:
## TODO Code here....
cluster = KMeans(n_clusters=4, random_state=RANDOM_SEED)
cluster.fit(X)
labels = cluster.predict(X)

### Task 1b

Try different numbers of clusters, compute the average silhouette scores using
`sklearn.metrics.silhouette_score` and plot the silhouette plot for the different values of K. Refer to the documentation: <https://scikit-learn.org/stable/modules/generated/sklearn.metrics.silhouette_score.html#sklearn.metrics.silhouette_score>



In [ ]:
from sklearn.metrics import silhouette_score

In [ ]:
## TODO Code here....
silhouette_scores = []
for k in range(2, 10):
  cluster = KMeans(n_clusters=k, random_state=RANDOM_SEED)
  cluster.fit(X)
  labels = cluster.predict(X)
  silhouette_scores.append(silhouette_score(X, labels))

plt.plot(range(2, 10), silhouette_scores)

### Task 1c

Select the number of clusters K using the silhouette method. Visualize the resulting clustering for in a scatter plot by using different colors for each cluster and also depicting the cluster centroids.

In [ ]:
## TODO Code here....
cluster = KMeans(n_clusters=4, random_state=RANDOM_SEED)
cluster.fit(X)
labels = cluster.predict(X)
centers = cluster.cluster_centers_

plt.scatter(X[:, 0], X[:, 1], c=labels
            , s=50)
plt.scatter(centers[:, 0], centers[:, 1], c="black", marker="x", s=25.)

## 📢 **HAND-IN** 📢:
1. The plot of the average silhouette scores for different values of K
2. The optimal K you selected based on the plots of the silhouette scores
3. The scatter plot of the clustering results depicting also the centroids for the optimal K

# TASK 2: DBSCAN

First, let's setup the data used in this section. We will redefine the variable `X` containing the data to be clustered.

In [ ]:
from sklearn.datasets import make_moons

In [ ]:
X, _ = make_moons(
    n_samples=200,
    noise=.05,
    random_state=RANDOM_SEED,
)

In [ ]:
plt.scatter(X[:, 0], X[:, 1], s=50)

### Task 2a

* cluster the new data `X` using `KMeans`
* set `n_clusters=2`
* visualize and analyse the resulting clustering
* What do you think of the result?

In [ ]:
## TODO Code here....
cluster = KMeans(n_clusters=2, random_state=RANDOM_SEED)
cluster.fit(X)
labels = cluster.predict(X)
centers = cluster.cluster_centers_
plt.scatter(X[:, 0], X[:, 1], c=labels
            , s=50)
plt.scatter(centers[:, 0], centers[:, 1], c="black", marker="x", s=25.)

### Task 2b

Let's use `sklearn.cluster.DBSCAN` instead. Refer to the documentation: <https://scikit-learn.org/stable/modules/generated/sklearn.cluster.DBSCAN.html>

* cluster the data using `DBSCAN`
* try different values for `eps` and `min_samples` until you manage to obtain good clustering of the two half-moons

In [ ]:
from sklearn.cluster import DBSCAN

In [ ]:
## TODO Code here....
dbscan = DBSCAN(eps=0.3, min_samples=5)
dbscan.fit(X)
labels = dbscan.labels_
plt.scatter(X[:, 0], X[:, 1], c=labels
            , s=50)

## 📢 **HAND-IN** 📢: No submission needed for this task.

# Task 3 (6 Points): Image Compression - Color Clustering in Images

K-Means can be used for image compression. This task focuses on appling this compression technique to an image.

We provide some useful helper functions below. Read the comments in the code carefully, but do not worry if you don't understand every line.


In [ ]:
from PIL import Image
import requests
from io import BytesIO

In [ ]:
def download_img(url: str) -> Image:
  """
  This function fetches an image from the internet and returns a PIL.Image object
  see: https://pillow.readthedocs.io/en/stable/reference/Image.html

  we tested it mainly on images from wikimedia
  """

  # have to set a fake user-agent so we dont get blocked by wikimedia
  headers = {'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/102.0.0.0 Safari/537.36'}
  r = requests.get(url, headers=headers)
  if r.status_code != 200:
    # if you hit this exception, consider using another image
    raise Exception(f"download failed:\n{url}")

  return Image.open(BytesIO(r.content)).convert("RGB")

def img2arr(img: Image) -> np.array:
  """
  convert a PIL.Image object to a numpy array
  the resulting array has 3 dimensions [height, width, 3]
  the last dimension contains rgb values

  the rgb values are normalized to be between 0. and 1.
  """
  return np.asarray(img) / 255

def arr2img(arr: np.array) -> Image:
  """
  convert a numpy array back into a PIL.Image object
  we expect the rgb values of the array to be between 0. and 1.
  """
  return Image.fromarray((arr * 255).astype(np.int8), mode="RGB")

In [ ]:
img_url = "https://upload.wikimedia.org/wikipedia/commons/d/d7/RGB_24bits_palette_sample_image.jpg"

img = download_img(img_url)

# you can visualize a PIL.Image object directly in jupyter using `display`
display(img)

img_arr = img2arr(img)

# visualize the np.array version of the same image
plt.imshow(img_arr)

In [ ]:
from typing import Optional

def rg_chromaticity(color_arr: np.array) ->  np.array:
  """
  helper function
  """
  sums = np.sum(color_arr, axis=1, keepdims=True)
  normed = np.divide(color_arr, sums, where=sums > 0.)
  return normed

def rg_chroma_plot(img_arr: np.array, centers: Optional[np.array] = None):
  """
  plot an image in rg-chromaticity space
  this is a 2D representation of 3D rgb data
  refer to wikipedia for details: https://en.wikipedia.org/wiki/Rg_chromaticity

  Note: the resulting plot will not accurately reflect the original euclidean distances

  inputs:
  img_arr: a numpy array with 3 dimensions [height, width, 3] representing an image
  centers: a numpy array with 2 dimensions [n_centers, 3] representing the cluster centers
  """
  colors = np.copy(img_arr).reshape((-1, 3))
  colors = np.unique(colors, axis=0)
  img_rg = rg_chromaticity(colors)
  plt.scatter(img_rg[:, 0], img_rg[:, 1], c=[tuple(colors[i]) for i in range(colors.shape[0])], s=.1)

  if centers is not None:
    crg = rg_chromaticity(centers)
    plt.scatter(crg[:, 0], crg[:, 1], c="black", marker="x", s=25.)

  plt.xlabel("red")
  plt.ylabel("green")
  plt.show()


In [ ]:
rg_chroma_plot(img_arr)

In [ ]:
from sklearn.metrics import pairwise_distances_argmin

def replace_nearest_color(img_arr: np.array, centers: np.array):
  """
  replace each pixel color in `img_arr` by the closest color in `centers`

  input:
  img_arr: a numpy array with 3 dimensions [height, width, 3] representing an image
  centers: a numpy array with 2 dimensions [n_centers, 3] representing the cluster centers
  """
  colors = img_arr.reshape((-1, 3))
  labels = pairwise_distances_argmin(colors, centers)
  compressed = labels.reshape(img_arr.shape[:2])
  replaced = centers[compressed]
  return replaced


In [ ]:
# generate 8 random colors for illustration
random_centers = np.random.default_rng(RANDOM_SEED).random(size=(8, 3))

# plot the random centers on top of the colors of the image
rg_chroma_plot(img_arr, random_centers)

# replace original colors by their nearest neighbors out of the candidate centers
replaced = replace_nearest_color(img_arr, random_centers)

# convert to PIL.Image and visualize
display(arr2img(replaced))

### Task 3 (continued)

* Use an image from [wikimedia](https://commons.wikimedia.org/wiki/Main_Page) to compress. Make sure that it is not too big, e.g. less than 1000px per side

* You can also use the same image we used in the example above

* download the image and convert it to a numpy array using the functions `download_img` and `img2arr` we defined above  

* Cluster the colors in the image using `KMeans`

* Choose the appropriate hyperparameters and verify them (e.g. using `silhouette_score` or the elbow method)

* Visualize your final cluster centers using `rg_chroma_plot` (see example usage above)

* Replace the colors of the original image by their nearest center using `replace_nearest_color` and display the result (see example usage above)

### Tips

* The images have 3 dimensions but the clustering algorithms expect 2. You can use [np.reshape](https://numpy.org/doc/stable/reference/generated/numpy.reshape.html) to obtain the dimension you need.
* To download the images you can right-click and save them to your computer.

In [ ]:
## TODO Code here....
img_url = "https://upload.wikimedia.org/wikipedia/commons/thumb/1/1f/Neorthacris_simulans-Kadavoor-2016-09-13-002.jpg/250px-Neorthacris_simulans-Kadavoor-2016-09-13-002.jpg"
img = download_img(img_url)
# you can visualize a PIL.Image object directly in jupyter using `display`
display(img)

img_arr = img2arr(img)

# visualize the np.array version of the same image
plt.imshow(img_arr)

## 📢 **HAND-IN** 📢:

* The original image you used for this task if different than the example image

* The final (compressed) image showing the replaced colors

* Your code for computing the clustering

* Short (2-3 sentences) description on how you verified the clustering and selected the hyperparameters


# Task
The user wants to compress an image using KMeans. To achieve this, I need to:
1. Reshape the image data from its 3D format to a 2D array where each row represents a pixel's RGB color.
2. Determine the optimal number of clusters (K) for KMeans by calculating and plotting silhouette scores for a range of K values.
3. Apply KMeans clustering to the reshaped image data using the identified optimal K, and extract the resulting cluster centers (the new color palette).
4. Visualize these cluster centers in rg-chromaticity space using the provided `rg_chroma_plot` function.
5. Compress the original image by replacing each pixel's color with its nearest color from the new palette using `replace_nearest_color`, and then display the compressed image.

## Reshape Image Data

### Subtask:
Reshape the `img_arr` from its original 3D format ([height, width, 3]) into a 2D array ([number_of_pixels, 3]) suitable for KMeans clustering. This array will represent each pixel's color as a data point.


**Reasoning**:
To prepare the image data for KMeans clustering, I need to reshape the 3D `img_arr` into a 2D array where each row represents a pixel and its RGB values. This is a standard preprocessing step for image compression using K-Means.



In [ ]:
height, width, _ = img_arr.shape
pixels = img_arr.reshape((height * width, 3))
print("Shape of the original image array:", img_arr.shape)
print("Shape of the reshaped pixels array:", pixels.shape)

## Determine Optimal K using Silhouette Score

### Subtask:
Determine the optimal number of clusters (K) for KMeans by calculating the average silhouette scores for a range of K values and plotting them. This will help in selecting an appropriate `n_clusters` for the KMeans model.

#### Instructions:
1. Iterate through a range of possible `n_clusters` values (e.g., from 2 to 10).
2. For each `k`:
    a. Create a `KMeans` instance with `n_clusters=k` and `random_state=RANDOM_SEED`.
    b. Fit the KMeans model to the `pixels` data.
    c. Predict the cluster `labels` for the `pixels` data.
    d. Calculate the `silhouette_score` using the `pixels` data and the obtained `labels`.
    e. Store the `k` and its corresponding `silhouette_score`.
3. Plot the silhouette scores against the number of clusters (`k`) to visualize the trend and identify the optimal K.

**Reasoning**:
I will calculate the silhouette scores for different numbers of clusters (K) from 2 to 10. For each K, I will fit a KMeans model to the reshaped pixel data, predict the cluster labels, and then compute the silhouette score. Finally, I will plot these scores to visualize the optimal K.



In [ ]:
height, width, _ = img_arr.shape
pixels = img_arr.reshape((height * width, 3))
silhouette_scores = []
k_values = range(2, 11) # Test K from 2 to 10

for k in k_values:
  kmeans = KMeans(n_clusters=k, random_state=RANDOM_SEED, n_init=10) # n_init for robust initialization
  kmeans.fit(pixels)
  labels = kmeans.predict(pixels)
  score = silhouette_score(pixels, labels)
  silhouette_scores.append(score)
  print(f"K={k}, Silhouette Score: {score:.4f}")

plt.figure(figsize=(10, 6))
plt.plot(k_values, silhouette_scores, marker='o')
plt.title('Silhouette Score for K-Means Clustering')
plt.xlabel('Number of Clusters (K)')
plt.ylabel('Silhouette Score')
plt.xticks(k_values)
plt.grid(True)
plt.show()

In [ ]:
optimal_k = 4 # Chosen based on the silhouette plot

kmeans_optimal = KMeans(n_clusters=optimal_k, random_state=RANDOM_SEED, n_init=10)
kmeans_optimal.fit(pixels)

centers = kmeans_optimal.cluster_centers_

print(f"KMeans with K={optimal_k} fitted. Number of cluster centers: {len(centers)}")

# Visualize the final cluster centers
rg_chroma_plot(img_arr, centers)

**Reasoning**:
The previous execution produced a `DeprecationWarning` related to the `mode` parameter in `Image.fromarray` within the `arr2img` function, and also `astype(np.int8)` is incorrect for pixel values which should be `np.uint8`. I will fix the `arr2img` function by changing the data type to `np.uint8` and removing the deprecated `mode` parameter.



**Reasoning**:
The `arr2img` function has been updated to address the deprecation warning and data type issue. Now, I will re-execute the code that applies KMeans with the optimal K, extracts the cluster centers, visualizes them, compresses the image, and displays the compressed image, using the corrected `arr2img` function.



In [ ]:
optimal_k = 4 # Chosen based on the silhouette plot

kmeans_optimal = KMeans(n_clusters=optimal_k, random_state=RANDOM_SEED, n_init=10)
kmeans_optimal.fit(pixels)

# Get the cluster centers (new color palette)
centers = kmeans_optimal.cluster_centers_

print(f"KMeans with K={optimal_k} fitted. Number of cluster centers: {len(centers)}")

# Visualize the final cluster centers
rg_chroma_plot(img_arr, centers)

# Replace original colors by their nearest neighbors out of the candidate centers
replaced_img_arr = replace_nearest_color(img_arr, centers)

# Convert to PIL.Image and visualize
compressed_img = arr2img(replaced_img_arr)
display(compressed_img)

## Final Task

### Subtask:
Summarize the process, including the chosen image, the optimal number of clusters (K), the method used for hyperparameter verification (silhouette score), and display the final compressed image along with the original if different from the example.


## Summary:

### Q&A
*   **What was the optimal number of clusters (K) chosen?**
    The optimal number of clusters chosen was K=4.
*   **What method was used for hyperparameter verification?**
    The silhouette score was used for hyperparameter verification to determine the optimal K.
*   **Was the final compressed image displayed?**
    Yes, the final compressed image was displayed successfully after the compression process.

### Data Analysis Key Findings
*   The input image, with an initial shape of (375, 250, 3), was successfully reshaped into a 2D array of pixels with dimensions (93750, 3) for KMeans clustering.
*   Silhouette scores were calculated for K values ranging from 2 to 10. The scores were: K=2: 0.8000, K=3: 0.7652, K=4: 0.7427, K=5: 0.6508, K=6: 0.4627, K=7: 0.4559, K=8: 0.4516, K=9: 0.4203, K=10: 0.4217.
*   K=4 was selected as the optimal number of clusters, offering a balance between a relatively high silhouette score (0.7427) and a reasonable number of colors for image compression.
*   KMeans clustering was applied with the optimal K=4, and the resulting four cluster centers (representing the new color palette) were extracted and visualized in rg-chromaticity space.
*   During the process, a `DeprecationWarning` related to the `arr2img` function (specifically `astype(np.int8)`) was identified and corrected by updating the function to use `astype(np.uint8)` and removing a deprecated `mode` parameter, ensuring correct image conversion and display.
*   The image was successfully compressed by replacing each pixel's original color with its nearest color from the 4-color palette.

### Insights or Next Steps
*   The selection of K=4, while based on silhouette score, represents a trade-off. Further subjective visual inspection could be performed to confirm if this K value provides the best perceived quality-to-compression ratio for different image types.
*   To potentially achieve even better compression or visual quality, future steps could explore alternative clustering algorithms, different color space transformations (e.g., CIELAB), or hybrid approaches that combine color quantization with other image compression techniques.
